<a target="_blank" href="https://colab.research.google.com/github/cohere-ai/notebooks/blob/main/notebooks/llmu/co_aws_ch3_text_generation.ipynb"> <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>

# Text Generation Using Cohere Command on Amazon Bedrock

Cohere's Command family of models, available on Amazon Bedrock, are powerful LLMs that offer these capabilities. This includes cutting-edge models like Command R and Command R+

In this notebook, we'll explore how to use Cohere's Command R+ model on Amazon Bedrock. We'll use a customer support agent scenario as an example and cover a range of tasks, including text generation, summarization, rewriting, and extraction.

# Setup

First, let's install and import the necessary libraries and set up our Cohere client using the cohere SDK. To use Bedrock, we create a BedrockClient by passing the necessary AWS credentials.

In [ ]:
! pip install cohere -q

In [2]:
import cohere

# Create Bedrock client via the native Cohere SDK
# Contact your AWS administrator for the credentials
co = cohere.BedrockClient(
    aws_region="YOUR_AWS_REGION",
    aws_access_key="YOUR_AWS_ACCESS_KEY_ID",
    aws_secret_key="YOUR_AWS_SECRET_ACCESS_KEY",
    aws_session_token="YOUR_AWS_SESSION_TOKEN",
)

# Creating some contextual information

Before we begin, let's create some context to use in our text generation tasks. In this example, we'll use a technical support FAQ as our context:

In [4]:
# Technical support FAQ
faq_tech_support = """- Question: How do I set up my new smartphone with my mobile plan?
- Answer:
  - Insert your SIM card into the device.
  - Turn on your phone and follow the on-screen setup instructions.
  - Connect to your mobile network and enter your account details when prompted.
  - Download and install any necessary apps or updates.
  - Contact customer support if you need further assistance.

- Question: My internet connection is slow. How can I improve my mobile data speed?
- Answer:
  - Check your signal strength and move to an area with better coverage.
  - Restart your device and try connecting again.
  - Ensure your data plan is active and has sufficient data.
  - Consider upgrading your plan for faster speeds.

- Question: I can't connect to my mobile network. What should I do?
- Answer:
  - Check your SIM card is inserted correctly and not damaged.
  - Restart your device and try connecting again.
  - Ensure your account is active and not suspended.
  - Check for any network outages in your area.
  - Contact customer support for further assistance.

- Question: How do I set up my voicemail?
- Answer:
  - Dial your voicemail access number (usually provided by your carrier).
  - Follow the prompts to set up your voicemail greeting and password.
  - Record your voicemail greeting and save it.
  - Test your voicemail by calling your number and leaving a message.

- Question: I'm having trouble sending text messages. What could be the issue?
- Answer:
  - Check your signal strength and move to an area with better coverage.
  - Ensure your account has sufficient credit or an active plan.
  - Restart your device and try sending a message again.
  - Check your message settings and ensure they are correct.
  - Contact customer support if the issue persists."""

# Function to generate text

Now, let's define a function to generate text using the Command R+ model on Bedrock.

In [17]:
def generate_text(message):
    response = co.chat(message=message,
                       model="cohere.command-r-plus-v1:0")
    return response

# Text generation

Let's use our technical support FAQ as context and generate a response to a customer inquiry.

In [18]:
inquiry = "I've noticed some fluctuations in my mobile network's performance recently.The connection seems stable most of the time, but every now and then, I experience brief periods of slow data speeds. It happens a few times a day and is quite inconvenient."

prompt = f"""Use the FAQs below to provide a concise response to this customer inquiry.

# Customer inquiry
{inquiry}

# FAQs
{faq_tech_support}"""

response = generate_text(prompt)

print(response.text)

Response to customer inquiry: 

Brief slowdowns in data speed can be frustrating. Here are some quick steps to improve your mobile data performance: 

1. Check your signal strength: Move to an area with better coverage if needed. 
2. Restart your device: A simple restart can often improve connectivity. 
3. Data plan status: Ensure your data plan is active and you have sufficient data allocated for faster speeds. 
4. Consider a plan upgrade: If the issue persists, consider contacting your provider to discuss faster speed plans. 

Additionally, checking for any network outages in your area and ensuring your SIM card is inserted correctly can also help maintain a stable connection. 

If the issue continues, reach out to your mobile provider's customer support for further guidance and troubleshooting.


# Text summarization

Now, let's summarize the customer inquiry into a single sentence.

In [19]:
prompt=f"""Summarize this customer inquiry into one short sentence.

Inquiry: {inquiry}"""

response = generate_text(prompt)

print(response.text)

I experience intermittent periods of slow data speeds a few times a day on my mobile network.


# Text Rewriting

Next, let's rewrite the generated response into an email format.

In [20]:
prompt=f"""Rewrite this customer support agent response into an email format, ready to send to the customer.

If you're experiencing brief periods of slow data speeds or difficulty sending text messages and connecting to your mobile network, here are some troubleshooting steps you can follow:

1. Check your signal strength - Move to an area with better coverage.
2. Restart your device and try connecting again.
3. Ensure your account is active and not suspended.
4. Contact customer support for further assistance. (This can include updating your plan for better network performance.)

Did these steps help resolve the issue? Let me know if you need further assistance."""

response = generate_text(prompt)

print(response.text)

Subject: Troubleshooting Guide for Slow Data and Network Issues

Dear [Customer Name], 

I hope this email finds you well. I understand that experiencing slow data speeds and network connectivity issues can be frustrating. Here are some detailed troubleshooting steps to help resolve these problems: 

- Signal Strength: 
   - Please check your device's signal strength by looking at the signal bars displayed on your screen. 
   - If you're in an area with weak coverage, try moving to a different location with better reception. You can also try going outdoors or near a window to improve signal strength. 

- Device Restart: 
   - Simply restart your device. This step can often resolve minor glitches and connectivity issues. Turn your device off, wait for a few seconds, and then turn it back on. 

- Account Status: 
   - Ensure that your account is active and in good standing. Sometimes, a suspended or inactive account can lead to connectivity issues. You can log in to your online account o

# Text Extraction

Now, let's extract some information from the customer inquiry and response.

In [24]:
prompt=f"""Extract the following as a JSON from the text provided below:
- category (Options: technical support, promotions, billing)
- product (Options: broadband, mobile, TV)
- status (Options: open, closed)

# Customer Inquiry
{inquiry}

# Customer Support Agent's Response
If you're experiencing brief periods of slow data speeds or difficulty sending text messages and connecting to your mobile network, here are some troubleshooting steps you can follow:

1. Check your signal strength - Move to an area with better coverage.
2. Restart your device and try connecting again.
3. Ensure your account is active and not suspended.
4. Contact customer support for further assistance. (This can include updating your plan for better network performance.)

Did these steps help resolve the issue? Let me know if you need further assistance."""

response = generate_text(prompt)

print(response.text)

```json
{
  "category": "technical support",
  "product": "mobile",
  "status": "open"
}
```


# Building a Chatbot

All our examples so far run on a single-turn interaction. But with the Chat endpoint, we can build a chatbot that keeps a memory of all previous interactions. This allows developers to build chatbot-style applications that maintain the state of a conversation.

The following is an implementation of a simple chatbot in a customer support setting, where the chatbot acts as a helpful customer support agent.

For this, we introduce a couple of additional parameters to the Chat endpoint:
- preamble: A preamble contains instructions to help steer a chatbot’s response toward specific characteristics, such as a persona, style, or format. Here we are using a simple preamble of “You are a helpful customer support agent that assist customers of a mobile network service.”
- chat_history: We store the history of a conversation between a user and the chatbot as a list, append every new conversation turn, and pass this information to the next endpoint call.

In [27]:
# Define a preamble
preamble = """## Task and Context
You are a helpful customer support agent that assists customers of a mobile network service."""

# Run the chatbot
def run_chatbot(message, chat_history=[]):
    response = co.chat(message=message,
                       model="cohere.command-r-plus-v1:0",
                       preamble=preamble,
                       chat_history=chat_history)
    
    print(response.text)
    
    chat_history = response.chat_history
            
    return chat_history

And here’s an example conversation that runs over a few turns.

In [28]:
chat_history = run_chatbot("Hi. I've noticed some fluctuations in my mobile network's performance recently.")

I'm sorry to hear that you've been experiencing issues with your mobile network's performance. Can you tell me a little more about the specific problems you've been facing? For example, are you experiencing dropped calls, slow data speeds, or something else entirely? The more information you can provide, the better I'll be able to assist you in troubleshooting the issue.


In [29]:
chat_history = run_chatbot("At times, the data speed is very poor. What should I do?", chat_history)

I'm sorry to hear that you're experiencing issues with your data speed. Here are a few troubleshooting steps you can try to improve your mobile data performance: 

- Check your device's signal: Weak or fluctuating signals can lead to poor data speeds. Try moving to a different location or adjusting your device's position to see if that improves your signal strength. 

- Restart your device: Sometimes, a simple restart can refresh your device's connection to the network and improve data speeds. 

- Check for network outages: It's possible that the issue is not on your end. Check your network provider's website or social media accounts to see if there are any reported outages or maintenance updates in your area. 

- Update your device software: Ensure that your device's operating system and apps are up to date. Outdated software can sometimes cause issues with network performance. 

- Review data usage and plan: Check your data usage to ensure you haven't exceeded your plan limits, which

In [30]:
chat_history = run_chatbot("Thanks, I'll try these.", chat_history)

You're welcome! If you have any other questions or concerns, feel free to reach out. I'm here to help.


Here's what is contained in the chat history after a few turns.

In [31]:
print("Chat history:")
for turn in chat_history:
    print(turn, "\n")

Chat history:
message="Hi. I've noticed some fluctuations in my mobile network's performance recently." tool_calls=None role='USER' 

message="I'm sorry to hear that you've been experiencing issues with your mobile network's performance. Can you tell me a little more about the specific problems you've been facing? For example, are you experiencing dropped calls, slow data speeds, or something else entirely? The more information you can provide, the better I'll be able to assist you in troubleshooting the issue." tool_calls=None role='CHATBOT' 

message='At times, the data speed is very poor. What should I do?' tool_calls=None role='USER' 

message="I'm sorry to hear that you're experiencing issues with your data speed. Here are a few troubleshooting steps you can try to improve your mobile data performance: \n\n- Check your device's signal: Weak or fluctuating signals can lead to poor data speeds. Try moving to a different location or adjusting your device's position to see if that imp

In this notebook, we explored how to use Command R+ on Amazon Bedrock for various text generation and manipulation tasks. We covered text generation, summarization, rewriting, and extraction, demonstrating how these capabilities can be applied to real-world scenarios.

If you want to learn more LLM use cases, visit our LLMU chapter on [use case patterns](https://docs.cohere.com/docs/use-case-patterns).